### Notebook to fetch files from azure blob storage


Data Model 

pre surgery

6 weeks
    patient<id>
        <assessment_time>
            left
            right
6 months 
    patient<id>

In [98]:
p_list = [
    (1, "665afc09824a6b1851b8c07f", 1), 
    (2, "665b03e0d7e141b790806601", 1),
    (3, "665b03f4d7e141b790806602", 1),
    (4, "665b0403d7e141b790806603", 1),
    (5, "665b0411d7e141b790806604", 1), 
    (6, "665d7aa6f472136d4f0327e6", 2), # under dir of walk pt6
    (7, "665ece3fe8995c510afe35b4", 2),
    (8, "6666a84bc5c65c1a0a814120", 2),
    (9, "6666a8ebc5c65c1a0a814122", 2),
    (10,"6666a864c5c65c1a0a814121", 2),
    (11,"6666a96ac5c65c1a0a814123", 3),
    (12, "666ffa646a8a304abfbbd386", 3),
    (13, "667016b74f7e24d69c766b06", 3),
    (14, "668d3cdcddb100ac4b670565", 3),
    (15, "668d4591979e85e6b46658b6", 3),
    (16, "669e187174ad33ebc71eb049", 3),
    (17, "66b9e96d9f4026cf251ff723", 1),
    (18,"66bb330314c767081f9c50a4",2 ),
    (19,"66bb4d4243348d3681fb0e8f", 3),
]


In [99]:
import azure_blob_storage as az_blob

In [100]:
# azure blob credentials - fetched on login as part of member api
credentials = {
    "sas_token": "sp=racwdli&st=2024-01-28T10:36:12Z&se=2024-12-31T18:36:12Z&spr=https&sv=2022-11-02&sr=c&sig=xYPJ3bPvganjajOiUMm8wcfqjYfg1oI6dCFbryrS9hE%3D",
    "storage_account_url": "https://rightstepdata.blob.core.windows.net",
    "container_name": "rs-data"
}

### Install / import the azure libraries

In [101]:
#!pip install azure-storage-blob 
#!pip install azure-identity
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient

#### 1. Create a blob service client

In [102]:
# this function should be in az_blob import
def blob_service_client(storage_account_url, sas_token):
    return BlobServiceClient(account_url=storage_account_url, credential=sas_token)

sc = blob_service_client(credentials["storage_account_url"], credentials["sas_token"])
#sc = az_blob.blob_service_client(credentials["storage_account_url"], credentials["sas_token"])
print(sc)

#### 2. Create a container client

In [103]:
def blob_container_client(blob_service_client, container_name):
    return blob_service_client.get_container_client(container=container_name)

bcc = blob_container_client(sc, credentials["container_name"])
print(bcc)

#### 3. Get the files to download

In [118]:
profile_id = "665afac88b3e08050604a8e6" # andy  <login_id><persin_id><timestamp><??>

def get_files_from_azure(profile_id, container_client):
    print("get files from azure", profile_id)
    path = profile_id
    files_to_download = container_client.list_blobs(path)
    return files_to_download

files = get_files_from_azure(profile_id, bcc  )
print(files)

get files from azure 665afac88b3e08050604a8e6
<iterator object azure.core.paging.ItemPaged at 0x25d1c018650>


### 4. Download all files for all patients

In [119]:
## TODO

# for each patient create directory for the patient i.e 1 that is in either grp 1,2,3
# create a directory for the timestamp 
# write the left and right data file here


# simplify syntax (ask chatgpt)
def get_patient_number(id, p_list):
    p_num = 0
    for num, id_str, grp in p_list:
        if(id_str == id):
            p_num = num
    return p_num

#'665afac88b3e08050604a8e6_665afc09824a6b1851b8c07f_1717404766904_D422CD005F87_walk_1_Right_Ankle_60.csv']
def download_files_from_azure_blob_storage(files_to_download, blob_service_client, container_name):
    for blob in files_to_download:
        blob_split = blob.name.split('/')
        p_id_to_check = blob_split[1]
        is_in_list = any(p_id_to_check == id_str for _, id_str, grp in p_list)
        if(is_in_list):
            index=0
            # patient_num_timestamp_walk1_left.csv
            file_name_split = blob_split[5].split("_")

            if("Left") in file_name_split:
                index  = file_name_split.index("Left")
            elif("Right") in file_name_split:
                index  = file_name_split.index("Right")
            
            patient_num = get_patient_number(p_id_to_check, p_list)
            local_file_name = str(patient_num) +"_" + file_name_split[2] + "_" + file_name_split[4] + "_" + file_name_split[index]+".csv"

            blob_client = blob_service_client.get_blob_client(container_name, blob, snapshot=None )
            with open(local_file_name, "wb") as my_blob:
                blob_data = blob_client.download_blob()
                blob_data.readinto(my_blob)

downloaded_files = download_files_from_azure_blob_storage(files, sc, credentials["container_name"])

INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://rightstepdata.blob.core.windows.net/rs-data?restype=REDACTED&comp=REDACTED&prefix=REDACTED&sp=REDACTED&st=REDACTED&se=REDACTED&spr=REDACTED&sv=REDACTED&sr=REDACTED&sig=REDACTED'
Request method: 'GET'
Request headers:
    'x-ms-version': 'REDACTED'
    'Accept': 'application/xml'
    'User-Agent': 'azsdk-python-storage-blob/12.23.0 Python/3.11.8 (Windows-10-10.0.22631-SP0)'
    'x-ms-date': 'REDACTED'
    'x-ms-client-request-id': 'ebaa8ffd-7bff-11ef-aacb-00090ffe0001'
No body was attached to the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/xml'
    'Server': 'Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0'
    'x-ms-request-id': 'fc6262f3-501e-0028-2d0c-108f6c000000'
    'x-ms-client-request-id': 'ebaa8ffd-7bff-11ef-aacb-00090ffe0001'
    'x-ms-version': 'REDACTED'
    'Date': 'Thu, 26 Sep 2

### Check patients id's have data files

In [92]:
# pTest 665d7aa6f472136d4f0327e6
# p1 665afc09824a6b1851b8c07f
# p2 665b03e0d7e141b790806601
# p3 665b03f4d7e141b790806602
# p4 665b0403d7e141b790806603
# p5 665b0411d7e141b790806604  
# p6 
# p7 665ece3fe8995c510afe35b4  or patient 6
# p8 6666a84bc5c65c1a0a814120
# p9 6666a8ebc5c65c1a0a814122
# p10 6666a864c5c65c1a0a814121
# p11 6666a96ac5c65c1a0a814123
# p12 666ffa646a8a304abfbbd386 or # this one isnt there anyway 666ffac26a8a304abfbbd387
# p13 667016b74f7e24d69c766b06   # doesnt exisit in data 668d3ca9ddb100ac4b670564 or 
# p14 668d3cdcddb100ac4b670565
# p15 668d4591979e85e6b46658b6
# p16 669e187174ad33ebc71eb049
# p17 66b9e96d9f4026cf251ff723
# p18 66bb330314c767081f9c50a4
# p19 66bb4d4243348d3681fb0e8f

# group 1 = pre
# group 2 = 6 weeks
# group 3 = 6 months

p_list = [
    (1, "665afc09824a6b1851b8c07f", 1), 
    (2, "665b03e0d7e141b790806601", 1),
    (3, "665b03f4d7e141b790806602", 1),
    (4, "665b0403d7e141b790806603", 1),
    (5, "665b0411d7e141b790806604", 1), 
    (6, "665d7aa6f472136d4f0327e6", 2), # under dir of walk pt6
    (7, "665ece3fe8995c510afe35b4", 2),
    (8, "6666a84bc5c65c1a0a814120", 2),
    (9, "6666a8ebc5c65c1a0a814122", 2),
    (10,"6666a864c5c65c1a0a814121", 2),
    (11,"6666a96ac5c65c1a0a814123", 3),
    (12, "666ffa646a8a304abfbbd386", 3),
    (13, "667016b74f7e24d69c766b06", 3),
    (14, "668d3cdcddb100ac4b670565", 3),
    (15, "668d4591979e85e6b46658b6", 3),
    (16, "669e187174ad33ebc71eb049", 3),
    (17, "66b9e96d9f4026cf251ff723", 1),
    (18,"66bb330314c767081f9c50a4",2 ),
    (19,"66bb4d4243348d3681fb0e8f", 3),
]

for blob in files:
    p_id_to_check = blob.name.split('/')[1]
    
    is_in_list = any(p_id_to_check == id_str for _, id_str in p_list)
    if(is_in_list):
        print("in list")
        #print(p_id_to_check, "in list")
    else:
        print(p_id_to_check, "not in list")

# check if this patient exists in our data set

# 665afac88b3e08050604a8e6/665afc09824a6b1851b8c07f/1717404766904/60/walk_1/665afac88b3e08050604a8e6_665afc09824a6b1851b8c07f_1717404766904_D422CD005F87_walk_1_Right_Ankle_60.csv
#['665afac88b3e08050604a8e6', '665afc09824a6b1851b8c07f', '1717404766904', '60', 'walk_1', '665afac88b3e08050604a8e6_665afc09824a6b1851b8c07f_1717404766904_D422CD005F87_walk_1_Right_Ankle_60.csv']

INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://rightstepdata.blob.core.windows.net/rs-data?restype=REDACTED&comp=REDACTED&prefix=REDACTED&sp=REDACTED&st=REDACTED&se=REDACTED&spr=REDACTED&sv=REDACTED&sr=REDACTED&sig=REDACTED'
Request method: 'GET'
Request headers:
    'x-ms-version': 'REDACTED'
    'Accept': 'application/xml'
    'User-Agent': 'azsdk-python-storage-blob/12.23.0 Python/3.11.8 (Windows-10-10.0.22631-SP0)'
    'x-ms-date': 'REDACTED'
    'x-ms-client-request-id': '24b8499c-7bfb-11ef-b7b6-00090ffe0001'
No body was attached to the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/xml'
    'Server': 'Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0'
    'x-ms-request-id': '9e54d268-a01e-004e-5c07-10c04c000000'
    'x-ms-client-request-id': '24b8499c-7bfb-11ef-b7b6-00090ffe0001'
    'x-ms-version': 'REDACTED'
    'Date': 'Thu, 26 Sep 2

in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
in list
